In [507]:
import numpy as np
import pandas as pd
import matplotlib as plt
import re

from pprint import pprint

#SKYLEARN
from sklearn.datasets import fetch_20newsgroups 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#NTLK
import nltk
nltk.download('stopwords')
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize, RegexpTokenizer
from nltk.tokenize import sent_tokenize, PunktSentenceTokenizer, WhitespaceTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn 
from nltk.corpus import stopwords

#CORPORA
import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dominic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [508]:

def preprocess( data ):
    newdata = []
    for idx, x in enumerate(data): #x -- document in data 
        newdoc = ""
        stop_words = stopwords.words('english')
    
        for token in sent_tokenize(x): #token -- sentence in doc
            seperator = " " 
            token = re.sub('[^A-Za-z0-9]+', ' ', token) #remove special characters
            token = re.sub(r'\d+', '', token) #remove numbers
            token = re.sub(r'\b\w{1,2}\b', '', token) #remove words with <= 2 characters
            whitespace_token = WhitespaceTokenizer().tokenize( token )
            wo_stopwords_token = [x for x in whitespace_token
                                  if not x in stop_words]
            newdoc += seperator.join( (wo_stopwords_token) ).lower()
            newdoc += " "                       
        data[idx] = newdoc     
    return data


In [509]:
def tf_idf( data ):
    tfidf_vectorizer = TfidfVectorizer(max_features=4000, analyzer='word', use_idf=True)
    tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(data)
  
    first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]

    df_idf = pd.DataFrame(tfidf_vectorizer.idf_, index=tfidf_vectorizer.get_feature_names(),columns=["idf_weights"])
    df_idf.sort_values(by=['idf_weights'], ascending=False)
    pprint(df_idf)

In [510]:
print("Loading 20 newsgroups dataset for categories:")
#pprint(list(newsgroups_train.target_names))

newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'))

pprint(newsgroups_train.data[102])

newdata = preprocess( newsgroups_train.data );

pprint("------ NEW DATA ---------")
pprint( newdata[102] )

#tf_idf( newdata )


Loading 20 newsgroups dataset for categories:
('/ hpcc01:rec.motorcycles / Stafford@Vax2.Winona.MSUS.Edu (John Stafford) / '
 '11:06 am  Apr  1, 1993 /\n'
 ' \n'
 '\n'
 '  It depreciates much faster, too.\n'
 '   \n'
 '====================================================\n'
 'John Stafford   Minnesota State University @ Winona\n'
 '                    All standard disclaimers apply.\n'
 '----------\n'
 "The '84 GL1200A hit the traps at 13.34 according to Cycle magazine. Yeah,\n"
 'they depreciate faster than Harleys for the first couple of years then\n'
 "they bottom out. Got my '86 GL1200I w/ 2275 miles on the odometer for\n"
 'just under $5K in May of 1990 and would ask for $4500 now with almost\n'
 "16K miles onnit....that's about 50% of what a new GL1500I would cost.\n"
 '\n'
 "Think the '86 GL1200I originally sold for $6500 brand new, not sure. \n"
 "If that's the case then it depreciated 30.77% over 7 years or a mere\n"
 '$2000. Big Fat Hairy Deal! Based on what I know, Harleys t